# Advent Of Code 2023

## Day 1: Trebuchet?!

In [2]:
# Part 1
def extract_number(from_line: str) -> int:
    clean_line = ''.join(char for char in from_line if char.isdigit())
    return int(clean_line[0] + clean_line[-1])

with open('files/day1-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all the calibration values is {sum(extract_number(line) for line in input_file)}")

The sum of all the calibration values is 55447


In [3]:
# Part 2
DIGIT_REPLACEMENT = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9
}

def extract_number(from_line: str) -> int:
    
    first_digit, last_digit = None, None

    for (index, char) in enumerate(from_line):
        
        # Match regular digit
        if char.isdigit():
            last_digit = char
        
        else:
            # Match spelled out digit
            matches = [value for digit, value in DIGIT_REPLACEMENT.items() if from_line[index:].startswith(digit)]
            if any(matches):
                last_digit = str(matches[0])
    
        if first_digit is None:
            first_digit = last_digit
            
    return int(first_digit + last_digit)

with open('files/day1-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all the calibration values is {sum(extract_number(line) for line in input_file)}")

The sum of all the calibration values is 54706


## Day 2 - Cube conundrum

In [51]:
# Part 1
import re

MAX_AMOUNTS = {
    "red":   12,
    "green": 13,
    "blue":  14,
}

RE_PARSE_GAME = re.compile(r"Game (\d+): (.*)")
RE_PARSE_DRAW = re.compile(r"(\d+) (red|green|blue)")

def process_game(line: str) -> tuple[int, int]:
    """Process the game result given as a string
    Returns a tuple containing the id of the game and True if the game is possible.
    """
    game_id, game_content = RE_PARSE_GAME.match(line).groups()

    return int(game_id), all(
        int(number) <= MAX_AMOUNTS[color]
        for number, color in (
            RE_PARSE_DRAW.search(bit).groups()
            for draw in game_content.split(sep=";")
            for bit in draw.split(sep=",")
        )
    )

with open('files/day2-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all possible game IDs is {sum(id for id, result in (process_game(line) for line in input_file) if result)}")

The sum of all possible game IDs is 2416


In [55]:
# Part 2
import re
import operator
from functools import reduce

RE_PARSE_GAME = re.compile(r"Game (\d+): (.*)")
RE_PARSE_DRAW = re.compile(r"(\d+) (red|green|blue)")

def process_game_power(line: str) -> int:
    """Process the game result given as a string
    Returns the power of the minimal possible set of cubes for the game.
    """
    game_id, game_content = RE_PARSE_GAME.match(line).groups()

    min_amounts = dict()
    for number, color in (RE_PARSE_DRAW.search(bit).groups() for draw in game_content.split(sep=";") for bit in draw.split(sep=",")):
        min_amounts[color] = max(int(number), min_amounts.get(color, 0))
        
    return reduce(operator.mul, min_amounts.values(), 1)

with open('files/day2-input', 'r', encoding='utf-8') as input_file:
    print(f"The sum of all possible game IDs is {sum(process_game_power(line) for line in input_file)}")

The sum of all possible game IDs is 63307
